In [ ]:
"""
Examples of numerical optimization algorithms
Based in part on code by Yiren Lu (https://github.com/yrlu/non-convex)
"""
import time
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
np.set_printoptions(precision=3, suppress=True)

# change parameters here
function_type = 0 #change this to 0 for biquadratic, 1 for Rosenbrock and 2 for Himmelblau function
starting_pos = np.array([1.2, 1.2])   #change starting position

# parameters that probably won't need changing
accuracy = 1e-4
max_iterations = 1000

# code for function definition
def function(x):
    if function_type == 0:
        return 100 * x[0]**2 + 200 * x[1]**2
    elif function_type == 1:
        return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
    elif function_type == 2:
        return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
    else:
        print("Unknown function_type: {}".format(function_type))
    
def gradient(x):
    if function_type == 0:
        return np.array([200 * x[0], 400 * x[1]])
    elif function_type == 1:
        return np.array([200*(x[1]-x[0]**2)*(-2*x[0]) + 2*(x[0]-1), 200*(x[1]-x[0]**2)])
    elif function_type == 2:
        return np.array([4 * x[0] * (x[0]**2 + x[1] - 11) + 2 * (x[0] + x[1]**2 - 7),
                         2 * (x[0]**2 + x[1] - 11) + 4 * x[1] * (x[0] + x[1]**2 - 7)])
    else:
        print("Unknown function_type: {}".format(function_type))

def hessian(x):
    if function_type == 0:
        return np.array([[200, 0], [0, 400]])
    elif function_type == 1:
        return np.array([[1200*x[0]**2 - 400*x[1] + 2, -400*x[0]], [-400*x[0], 200]])
    elif function_type == 2:
        return np.array([[4 * (x[0]**2 + x[1] - 11) + 8 * x[0]**2 + 2, 4 * x[0] + 4 * x[1]], 
                         [4 * x[0] + 4 * x[1], 4 * (x[0] + x[1]**2 -  7) + 8 * x[1]**2 + 2]])    
    else:
        print("Unknown function_type: {}".format(function_type))
        
# code for line search
def wolfe(f, g, xk, alpha, pk):
    c1 = 1e-4
    return f(xk + alpha * pk) <= f(xk) + c1 * alpha * np.dot(g(xk), pk)

def strong_wolfe(f, g, xk, alpha, pk, c2):
    # typically, c2 = 0.9 when using Newton or quasi-Newton's method.
    #            c2 = 0.1 when using non-linear conjugate gradient method.
    return wolfe(f, g, xk, alpha, pk) and abs(np.dot(g(xk + alpha * pk), pk)) <= c2 * abs(np.dot(g(xk), pk))

def step_length(f, g, xk, alpha, pk, c2):
    return interpolation(f, g,
                         lambda alpha: f(xk + alpha * pk),
                         lambda alpha: np.dot(g(xk + alpha * pk), pk),
                         alpha, c2,
                         lambda f, g, alpha, c2: strong_wolfe(f, g, xk, alpha, pk, c2))

def interpolation(f, g, f_alpha, g_alpha, alpha, c2, strong_wolfe_alpha, iters=20):
  # referred implementation here:
  # https://github.com/tamland/non-linear-optimization
    l = 0.0
    h = 1.0
    for i in range(iters):
        if strong_wolfe_alpha(f, g, alpha, c2):
            return alpha

        half = (l + h) / 2
        alpha = - g_alpha(l) * (h**2) / (2 * (f_alpha(h) - f_alpha(l) - g_alpha(l) * h))
        if alpha < l or alpha > h:
            alpha = half
        if g_alpha(alpha) > 0:
            h = alpha
        elif g_alpha(alpha) <= 0:
            l = alpha
    return alpha

# code for optimization algorithms
def steepest_descent(f, grad, x0, iterations, error):
    xhist = [x0]
    x = x0
    x_old = x
    c2 = 0.9
    for i in range(iterations):
        pk = -grad(x)
        alpha = step_length(f, grad, x, 1.0, pk, c2)
        x = x + alpha * pk
        xhist.append(x)
        
        if np.linalg.norm(x - x_old) < error:
            break
        x_old = x
        
    return xhist, i

def conjugate_gradient(f, g, x0, iterations, error):
    xhist = [x0]
    xk = x0
    c2 = 0.1

    fk = f(xk)
    gk = g(xk)
    pk = -gk

    for i in range(iterations):
        alpha = step_length(f, g, xk, 1.0, pk, c2)
        xk1 = xk + alpha * pk
        gk1 = g(xk1)
        beta_k1 = np.dot(gk1, gk1) / np.dot(gk, gk)
        pk1 = -gk1 + beta_k1 * pk
        xhist.append(xk1)
        
        if np.linalg.norm(xk1 - xk) < error:
            xk = xk1
            break

        xk = xk1
        gk = gk1
        pk = pk1

    return xhist, i

def newton(f, g, H, x0, iterations, error):
    xhist = [x0]
    x = x0
    x_old = x
    c2 = 0.9
    for i in range(iterations):
        pk = -np.linalg.solve(H(x), g(x))
        alpha = step_length(f, g, x, 1.0, pk, c2)
        x = x + alpha * pk
        xhist.append(x)
        
        if np.linalg.norm(x - x_old) < error:
            break
        x_old = x
        
    return xhist, i

def bfgs(f, g, x0, iterations, error):
    xhist = [x0]
    xk = x0
    c2 = 0.9
    I = np.identity(xk.size)
    Hk = I

    for i in range(iterations):
        # compute search direction
        gk = g(xk)
        pk = -Hk.dot(gk)

        # obtain step length by line search
        alpha = step_length(f, g, xk, 1.0, pk, c2)

        # update x
        xk1 = xk + alpha * pk
        gk1 = g(xk1)

        # define sk and yk for convenience
        sk = xk1 - xk
        yk = gk1 - gk

        # compute H_{k+1} by BFGS update
        rho_k = float(1.0 / yk.dot(sk))

        Hk1 = (I - rho_k * np.outer(sk, yk)).dot(Hk).dot(I - \
               rho_k * np.outer(yk, sk)) + rho_k * np.outer(sk, sk)
        
        if np.linalg.norm(xk1 - xk) < error:
            xk = xk1
            xhist.append(xk1)
            break

        Hk = Hk1
        xk = xk1
        xhist.append(xk1)

    return xhist, i

# plot function
def npmap2d(fun, xs, ys):
    Z = np.empty(len(xs) * len(ys))
    i = 0
    for y in ys:
        for x in xs:
            Z[i] = fun(np.array([x, y]))
            i += 1
    X, Y = np.meshgrid(xs, ys)
    Z.shape = X.shape
    return X, Y, Z

fig = plt.figure(figsize=(20,15))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.view_init(azim=0, elev=90)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')
if function_type == 0:
    step = 0.1
    X = np.arange(-2., 2.+step, step)
    Y = np.arange(-2., 2.+step, step)
    zmin = 0
    zmax = 600
elif function_type == 1:
    step = 0.1
    X = np.arange(-2.5, 2.5+step, step)
    Y = np.arange(-2., 4.+step, step)
elif function_type == 2:
    step = 0.2
    X = np.arange(-6., 6.+step, step)
    Y = np.arange(-6., 6.+step, step)
    zmin = 0
    zmax = 200
else:
    print("Unknown function type: {}".format(function_type))
X, Y, Z = npmap2d(function, X, Y)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=plt.cm.coolwarm, vmin = zmin, vmax = zmax, linewidth=0, antialiased=True, alpha=0.5)
fig.colorbar(surf)

start = time.time()
x, n_iter = steepest_descent(function, gradient, starting_pos, iterations=max_iterations, error=accuracy)
end = time.time()
print("Steepest Descent  \n    {:4d} iterations, minimum = ({:.2f}, {:.2f}), f={:.2f}, time elapsed {:.4f} sec, secs per iteration {:.6f}".format(n_iter, x[-1][0], x[-1][1], function(x[-1]), end - start, (end - start) / n_iter))
ax.scatter([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='r', label='Steepest Descent')
ax.plot3D([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='r')

start = time.time()
x, n_iter = conjugate_gradient(function, gradient, starting_pos, iterations=max_iterations, error=accuracy)
end = time.time()
print("Conjugate Gradient\n    {:4d} iterations, minimum = ({:.2f}, {:.2f}), f={:.2f}, time elapsed {:.4f} sec, secs per iteration {:.6f}".format(n_iter, x[-1][0], x[-1][1], function(x[-1]), end - start, (end - start) / n_iter))
ax.scatter([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='g', label='Conjugate Gradient')
ax.plot3D([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='g')

start = time.time()
x, n_iter = newton(function, gradient, hessian, starting_pos, iterations=max_iterations, error=accuracy)
end = time.time()
print("Newton            \n    {:4d} iterations, minimum = ({:.2f}, {:.2f}), f={:.2f}, time elapsed {:.4f} sec, secs per iteration {:.6f}".format(n_iter, x[-1][0], x[-1][1], function(x[-1]), end - start, (end - start) / n_iter))
ax.scatter([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='b', label='Newton')
ax.plot3D([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='b')

start = time.time()
x, n_iter = bfgs(function, gradient, starting_pos, iterations=max_iterations, error=accuracy)
end = time.time()
print("BFGS              \n    {:4d} iterations, minimum = ({:.2f}, {:.2f}), f={:.2f}, time elapsed {:.4f} sec, secs per iteration {:.6f}".format(n_iter, x[-1][0], x[-1][1], function(x[-1]), end - start, (end - start) / n_iter))
ax.scatter([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='purple', label='BFGS')
ax.plot3D([t[0] for t in x], [t[1] for t in x], [np.max(Z)]*len(x), c='purple')

l = ax.legend()